In [27]:

import pandas as pd
import numpy as np 
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc,accuracy_score,f1_score,precision_score,recall_score,classification_report
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import precision_score,recall_score,auc

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_data = pd.read_csv("new_features.csv")
train_labels = pd.read_csv("train_labels.csv")
test_labels = pd.read_csv("new_test.csv")

In [4]:
train_labels['building_id'].equals(train_data['building_id'])

True

In [5]:
#building_damage = train_data.merge(train_labels, how = 'inner', on = 'building_id')
building_damage = train_data
building_damage['damage_grade']

0         3
1         2
2         3
3         2
4         3
         ..
260596    2
260597    3
260598    3
260599    2
260600    3
Name: damage_grade, Length: 260601, dtype: int64

In [6]:
test_labels.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,geo_level_1id_new,geo_level_2id_new,geo_level_3id_new,superstructure_first_combination,superstructure_second_combination
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,1,136.0,388.0,1,0
1,99355,6,141,11987,2,25,13,5,t,r,...,0,0,0,0,0,15,520.0,1328.0,1,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,25,525.0,1.0,1,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,0,0,0,31,1032.0,1786.0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,1,142.0,191.0,1,0


In [7]:
test_labels['damage_grade'] = -1
building_damage = building_damage.append(test_labels, ignore_index=True, sort=False)

In [8]:
building_damage.drop_duplicates(inplace=True)
building_damage = building_damage.drop(columns ="has_secondary_use")

In [9]:
building_damage.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,geo_level_1id_new,geo_level_2id_new,geo_level_3id_new,superstructure_first_combination,superstructure_second_combination
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,3,15,122.0,176.0,1,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,2,4,243.0,1243.0,1,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,3,3,228.0,454.0,1,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,2,25,636.0,1187.0,1,1
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,3,8,353.0,737.0,1,0


In [10]:
#import featuretools as ft

In [11]:
#es = ft.EntitySet(id = 'earthquake')

# adding a dataframe 
#es.add_dataframe(dataframe_name = 'training_set', dataframe = building_damage, index = 'building_id')

In [12]:
#feature_matrix, feature_names = ft.dfs(entityset=es, 
#target_dataframe_name = 'training_set', 
#max_depth = 10, 
#verbose = 1, 
#n_jobs = 1)

In [13]:
#feature_matrix.columns

In [14]:
train_final = building_damage

Geo lebel 1 id 

In [107]:
damage1 = {}
damage2 = {}
damage3 = {}
for i,j in (train_final['geo_level_1_id'].value_counts()).iteritems():
    n1 = len(train_final[train_final['damage_grade']==1][train_final['geo_level_1_id']==i])
    n2 = len(train_final[train_final['damage_grade']==2][train_final['geo_level_1_id']==i])
    n3 = len(train_final[train_final['damage_grade']==3][train_final['geo_level_1_id']==i])
    
    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
list1 = []
list2 = []
list3 = []
for i in train_final['geo_level_1_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))
train_final['prob1_geo1'] = list1
train_final['prob2_geo1'] = list2
train_final['prob3_geo1'] = list3

Geo lebel 2 id

In [109]:
damage1 = {}
damage2 = {}
damage3 = {}

for i,j in (train_final['geo_level_2_id'].value_counts()).iteritems():
    
    n1 = len(train_final[train_final['damage_grade']==1][train_final['geo_level_2_id']==i])
    n2 = len(train_final[train_final['damage_grade']==2][train_final['geo_level_2_id']==i])
    n3 = len(train_final[train_final['damage_grade']==3][train_final['geo_level_2_id']==i])

    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
    
list1 = []
list2 = []
list3 = []

for i in train_final['geo_level_2_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

train_final['prob1_geo2'] = list1
train_final['prob2_geo2'] = list2
train_final['prob3_geo2'] = list3

Geo lebel 3 id

In [110]:
damage1 = {}
damage2 = {}
damage3 = {}

for i,j in (train_final['geo_level_3_id'].value_counts()).iteritems():
    
    n1 = len(train_final[train_final['damage_grade']==1][train_final['geo_level_3_id']==i])
    n2 = len(train_final[train_final['damage_grade']==2][train_final['geo_level_3_id']==i])
    n3 = len(train_final[train_final['damage_grade']==3][train_final['geo_level_3_id']==i])

    damage1[i] = n1/j
    damage2[i] = n2/j
    damage3[i] = n3/j
    
list1 = []
list2 = []
list3 = []

for i in train_final['geo_level_3_id']:
    list1.append(damage1.get(i))
    list2.append(damage2.get(i))
    list3.append(damage3.get(i))

train_final['prob1_geo3'] = list1
train_final['prob2_geo3'] = list2
train_final['prob3_geo3'] = list3

In [15]:
train_final.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,geo_level_1id_new,geo_level_2id_new,geo_level_3id_new,superstructure_first_combination,superstructure_second_combination
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,3,15,122.0,176.0,1,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,2,4,243.0,1243.0,1,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,3,3,228.0,454.0,1,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,2,25,636.0,1187.0,1,1
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,3,8,353.0,737.0,1,0


In [112]:
train_final.to_csv('train_final_featured.csv',index=False)

In [16]:
from catboost import CatBoostRegressor

categorical_features = np.where(train_final.dtypes == 'object')[0]
print(categorical_features)
for i in categorical_features:
    train_final.iloc[:,i] = train_final.iloc[:,i].astype('str')

[ 8  9 10 11 12 13 14 22]


In [17]:
train_final.iloc[:,1] = train_final.iloc[:,1].astype('str')
train_final.iloc[:,2] = train_final.iloc[:,2].astype('str')
train_final.iloc[:,3] = train_final.iloc[:,3].astype('str')

In [18]:
train = train_final[(train_final['damage_grade'] != -1)]
train.drop('building_id',axis=1)
test = train_final[(train_final['damage_grade'] == -1)]
test.index -=260601

In [19]:
from sklearn.model_selection import train_test_split
X = train.drop('damage_grade',axis=1)
y = train['damage_grade']


In [20]:
categorical_features = X.columns[np.where(train_final.dtypes  == 'object')[0]].values.tolist()
print(categorical_features)

['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=35)

In [22]:
from catboost import CatBoostClassifier
model_cat = CatBoostClassifier(iterations=2000, learning_rate=0.04, depth=8, eval_metric='TotalF1',loss_function='MultiClass', random_seed=32)
#task_type ="GPU" try 5000, learning 0.05
# training model
model_cat.fit(X_train, y_train, cat_features=categorical_features)

# validation score
model_cat.score(X_test, y_test)


0:	learn: 0.7257236	total: 845ms	remaining: 28m 8s
1:	learn: 0.7332230	total: 1.6s	remaining: 26m 39s
2:	learn: 0.7282066	total: 2.34s	remaining: 26m
3:	learn: 0.7295087	total: 3.07s	remaining: 25m 31s
4:	learn: 0.7287540	total: 3.87s	remaining: 25m 43s
5:	learn: 0.7336599	total: 4.63s	remaining: 25m 37s
6:	learn: 0.7342655	total: 5.41s	remaining: 25m 39s
7:	learn: 0.7348552	total: 6.18s	remaining: 25m 38s
8:	learn: 0.7346613	total: 7.13s	remaining: 26m 16s
9:	learn: 0.7351188	total: 7.92s	remaining: 26m 16s
10:	learn: 0.7346699	total: 8.47s	remaining: 25m 31s
11:	learn: 0.7354461	total: 9.3s	remaining: 25m 41s
12:	learn: 0.7356080	total: 10.1s	remaining: 25m 38s
13:	learn: 0.7357049	total: 10.8s	remaining: 25m 34s
14:	learn: 0.7362364	total: 11.6s	remaining: 25m 38s
15:	learn: 0.7364083	total: 12.4s	remaining: 25m 31s
16:	learn: 0.7364433	total: 13.2s	remaining: 25m 34s
17:	learn: 0.7367396	total: 13.9s	remaining: 25m 26s
18:	learn: 0.7367289	total: 14.7s	remaining: 25m 28s
19:	learn:

154:	learn: 0.7518921	total: 2m 27s	remaining: 29m 12s
155:	learn: 0.7519642	total: 2m 28s	remaining: 29m 13s
156:	learn: 0.7520766	total: 2m 29s	remaining: 29m 14s
157:	learn: 0.7521014	total: 2m 30s	remaining: 29m 16s
158:	learn: 0.7521972	total: 2m 31s	remaining: 29m 18s
159:	learn: 0.7522124	total: 2m 32s	remaining: 29m 17s
160:	learn: 0.7522993	total: 2m 33s	remaining: 29m 16s
161:	learn: 0.7523465	total: 2m 35s	remaining: 29m 19s
162:	learn: 0.7523601	total: 2m 36s	remaining: 29m 19s
163:	learn: 0.7524485	total: 2m 37s	remaining: 29m 21s
164:	learn: 0.7525499	total: 2m 38s	remaining: 29m 23s
165:	learn: 0.7527432	total: 2m 39s	remaining: 29m 22s
166:	learn: 0.7527543	total: 2m 40s	remaining: 29m 23s
167:	learn: 0.7528251	total: 2m 41s	remaining: 29m 24s
168:	learn: 0.7528402	total: 2m 42s	remaining: 29m 23s
169:	learn: 0.7528972	total: 2m 43s	remaining: 29m 24s
170:	learn: 0.7529137	total: 2m 45s	remaining: 29m 25s
171:	learn: 0.7529283	total: 2m 46s	remaining: 29m 27s
172:	learn

304:	learn: 0.7585672	total: 5m 14s	remaining: 29m 7s
305:	learn: 0.7585507	total: 5m 15s	remaining: 29m 7s
306:	learn: 0.7586243	total: 5m 16s	remaining: 29m 5s
307:	learn: 0.7586374	total: 5m 17s	remaining: 29m 5s
308:	learn: 0.7586703	total: 5m 18s	remaining: 29m 3s
309:	learn: 0.7587151	total: 5m 19s	remaining: 29m 3s
310:	learn: 0.7587723	total: 5m 20s	remaining: 29m 2s
311:	learn: 0.7588023	total: 5m 21s	remaining: 29m
312:	learn: 0.7587982	total: 5m 22s	remaining: 28m 59s
313:	learn: 0.7588076	total: 5m 24s	remaining: 28m 59s
314:	learn: 0.7588127	total: 5m 24s	remaining: 28m 57s
315:	learn: 0.7588963	total: 5m 25s	remaining: 28m 56s
316:	learn: 0.7589521	total: 5m 26s	remaining: 28m 55s
317:	learn: 0.7589393	total: 5m 27s	remaining: 28m 54s
318:	learn: 0.7589932	total: 5m 28s	remaining: 28m 52s
319:	learn: 0.7590625	total: 5m 29s	remaining: 28m 52s
320:	learn: 0.7591002	total: 5m 31s	remaining: 28m 51s
321:	learn: 0.7591084	total: 5m 32s	remaining: 28m 50s
322:	learn: 0.7591172

455:	learn: 0.7626177	total: 7m 56s	remaining: 26m 53s
456:	learn: 0.7626488	total: 7m 57s	remaining: 26m 51s
457:	learn: 0.7627006	total: 7m 58s	remaining: 26m 50s
458:	learn: 0.7628517	total: 7m 59s	remaining: 26m 48s
459:	learn: 0.7628272	total: 8m	remaining: 26m 47s
460:	learn: 0.7628876	total: 8m 1s	remaining: 26m 46s
461:	learn: 0.7629377	total: 8m 2s	remaining: 26m 45s
462:	learn: 0.7629709	total: 8m 3s	remaining: 26m 45s
463:	learn: 0.7630031	total: 8m 4s	remaining: 26m 44s
464:	learn: 0.7630428	total: 8m 5s	remaining: 26m 43s
465:	learn: 0.7630319	total: 8m 6s	remaining: 26m 43s
466:	learn: 0.7630444	total: 8m 8s	remaining: 26m 42s
467:	learn: 0.7630627	total: 8m 9s	remaining: 26m 42s
468:	learn: 0.7631205	total: 8m 10s	remaining: 26m 40s
469:	learn: 0.7631393	total: 8m 11s	remaining: 26m 39s
470:	learn: 0.7631794	total: 8m 12s	remaining: 26m 38s
471:	learn: 0.7632669	total: 8m 13s	remaining: 26m 37s
472:	learn: 0.7632353	total: 8m 14s	remaining: 26m 35s
473:	learn: 0.7632413	

605:	learn: 0.7657004	total: 10m 43s	remaining: 24m 39s
606:	learn: 0.7657586	total: 10m 44s	remaining: 24m 39s
607:	learn: 0.7657950	total: 10m 45s	remaining: 24m 38s
608:	learn: 0.7658889	total: 10m 46s	remaining: 24m 37s
609:	learn: 0.7659114	total: 10m 48s	remaining: 24m 36s
610:	learn: 0.7659040	total: 10m 49s	remaining: 24m 36s
611:	learn: 0.7658897	total: 10m 50s	remaining: 24m 35s
612:	learn: 0.7659416	total: 10m 51s	remaining: 24m 34s
613:	learn: 0.7659569	total: 10m 52s	remaining: 24m 33s
614:	learn: 0.7659571	total: 10m 53s	remaining: 24m 32s
615:	learn: 0.7659967	total: 10m 55s	remaining: 24m 32s
616:	learn: 0.7660076	total: 10m 56s	remaining: 24m 31s
617:	learn: 0.7660548	total: 10m 57s	remaining: 24m 30s
618:	learn: 0.7660443	total: 10m 58s	remaining: 24m 28s
619:	learn: 0.7660933	total: 10m 59s	remaining: 24m 27s
620:	learn: 0.7660961	total: 11m	remaining: 24m 26s
621:	learn: 0.7661170	total: 11m 1s	remaining: 24m 25s
622:	learn: 0.7660830	total: 11m 2s	remaining: 24m 23

...

1484:	learn: 0.7801606	total: 30m 4s	remaining: 10m 25s
1485:	learn: 0.7801825	total: 30m 5s	remaining: 10m 24s
1486:	learn: 0.7801924	total: 30m 6s	remaining: 10m 23s
1487:	learn: 0.7802127	total: 30m 8s	remaining: 10m 22s
1488:	learn: 0.7802283	total: 30m 9s	remaining: 10m 21s
1489:	learn: 0.7802280	total: 30m 10s	remaining: 10m 19s
1490:	learn: 0.7802935	total: 30m 12s	remaining: 10m 18s
1491:	learn: 0.7802905	total: 30m 13s	remaining: 10m 17s
1492:	learn: 0.7802927	total: 30m 15s	remaining: 10m 16s
1493:	learn: 0.7803239	total: 30m 17s	remaining: 10m 15s
1494:	learn: 0.7803507	total: 30m 18s	remaining: 10m 14s
1495:	learn: 0.7803859	total: 30m 20s	remaining: 10m 13s
1496:	learn: 0.7803852	total: 30m 21s	remaining: 10m 12s
1497:	learn: 0.7803508	total: 30m 23s	remaining: 10m 11s
1498:	learn: 0.7803935	total: 30m 24s	remaining: 10m 9s
1499:	learn: 0.7804521	total: 30m 26s	remaining: 10m 8s
1500:	learn: 0.7804880	total: 30m 28s	remaining: 10m 7s
1501:	learn: 0.7804846	total: 30m 29s	r

1632:	learn: 0.7821311	total: 33m 23s	remaining: 7m 30s
1633:	learn: 0.7821442	total: 33m 24s	remaining: 7m 29s
1634:	learn: 0.7821352	total: 33m 26s	remaining: 7m 27s
1635:	learn: 0.7821561	total: 33m 27s	remaining: 7m 26s
1636:	learn: 0.7821389	total: 33m 28s	remaining: 7m 25s
1637:	learn: 0.7821492	total: 33m 29s	remaining: 7m 24s
1638:	learn: 0.7821700	total: 33m 30s	remaining: 7m 22s
1639:	learn: 0.7821848	total: 33m 31s	remaining: 7m 21s
1640:	learn: 0.7822067	total: 33m 33s	remaining: 7m 20s
1641:	learn: 0.7822677	total: 33m 34s	remaining: 7m 19s
1642:	learn: 0.7823002	total: 33m 35s	remaining: 7m 17s
1643:	learn: 0.7822729	total: 33m 36s	remaining: 7m 16s
1644:	learn: 0.7822809	total: 33m 38s	remaining: 7m 15s
1645:	learn: 0.7823454	total: 33m 39s	remaining: 7m 14s
1646:	learn: 0.7823320	total: 33m 40s	remaining: 7m 13s
1647:	learn: 0.7823336	total: 33m 41s	remaining: 7m 11s
1648:	learn: 0.7823300	total: 33m 42s	remaining: 7m 10s
1649:	learn: 0.7823217	total: 33m 44s	remaining:

1780:	learn: 0.7839812	total: 36m 17s	remaining: 4m 27s
1781:	learn: 0.7839820	total: 36m 19s	remaining: 4m 26s
1782:	learn: 0.7839820	total: 36m 20s	remaining: 4m 25s
1783:	learn: 0.7839802	total: 36m 21s	remaining: 4m 24s
1784:	learn: 0.7839792	total: 36m 22s	remaining: 4m 22s
1785:	learn: 0.7840095	total: 36m 23s	remaining: 4m 21s
1786:	learn: 0.7840623	total: 36m 24s	remaining: 4m 20s
1787:	learn: 0.7840967	total: 36m 25s	remaining: 4m 19s
1788:	learn: 0.7841144	total: 36m 26s	remaining: 4m 17s
1789:	learn: 0.7840831	total: 36m 27s	remaining: 4m 16s
1790:	learn: 0.7840902	total: 36m 28s	remaining: 4m 15s
1791:	learn: 0.7841001	total: 36m 29s	remaining: 4m 14s
1792:	learn: 0.7841336	total: 36m 31s	remaining: 4m 12s
1793:	learn: 0.7841675	total: 36m 32s	remaining: 4m 11s
1794:	learn: 0.7842386	total: 36m 33s	remaining: 4m 10s
1795:	learn: 0.7842332	total: 36m 34s	remaining: 4m 9s
1796:	learn: 0.7842283	total: 36m 35s	remaining: 4m 7s
1797:	learn: 0.7842396	total: 36m 36s	remaining: 4

1928:	learn: 0.7856486	total: 39m 15s	remaining: 1m 26s
1929:	learn: 0.7856275	total: 39m 16s	remaining: 1m 25s
1930:	learn: 0.7856283	total: 39m 17s	remaining: 1m 24s
1931:	learn: 0.7856257	total: 39m 18s	remaining: 1m 23s
1932:	learn: 0.7856293	total: 39m 19s	remaining: 1m 21s
1933:	learn: 0.7856390	total: 39m 21s	remaining: 1m 20s
1934:	learn: 0.7856211	total: 39m 22s	remaining: 1m 19s
1935:	learn: 0.7856039	total: 39m 23s	remaining: 1m 18s
1936:	learn: 0.7856218	total: 39m 24s	remaining: 1m 16s
1937:	learn: 0.7856347	total: 39m 26s	remaining: 1m 15s
1938:	learn: 0.7856670	total: 39m 27s	remaining: 1m 14s
1939:	learn: 0.7857262	total: 39m 28s	remaining: 1m 13s
1940:	learn: 0.7857181	total: 39m 29s	remaining: 1m 12s
1941:	learn: 0.7857190	total: 39m 30s	remaining: 1m 10s
1942:	learn: 0.7857326	total: 39m 31s	remaining: 1m 9s
1943:	learn: 0.7857334	total: 39m 32s	remaining: 1m 8s
1944:	learn: 0.7857305	total: 39m 34s	remaining: 1m 7s
1945:	learn: 0.7857495	total: 39m 35s	remaining: 1m

0.7766777944054334

In [25]:
# Predict for test dataset
#test = test.drop('damage_grade',axis=1)
prediction=model_cat.predict(test).tolist()
y_pred = model_cat .predict(X_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Accuracy: 0.7766777944054334
              precision    recall  f1-score   support

           1       0.73      0.59      0.65      2507
           2       0.77      0.87      0.82     14762
           3       0.80      0.68      0.73      8792

    accuracy                           0.78     26061
   macro avg       0.77      0.71      0.73     26061
weighted avg       0.78      0.78      0.77     26061



final 0.7443

In [29]:
result=pd.DataFrame(prediction)
result['building_id']=test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,3
86865,1049160,2
86866,442785,2


In [30]:
result['damage_grade'].value_counts()

2    55114
3    25020
1     6734
Name: damage_grade, dtype: int64

In [31]:
result.to_csv('result.csv',index=False)

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
cat_features = X_train.columns[np.where(X_train.dtypes != float)[0]].values.tolist()
X_train[cat_features] = X_train[cat_features].astype(str)

In [34]:
parameters = {'depth'         : [7,8,9, 10],
                 'learning_rate' : [0.04, 0.05, 0.06, 0.07, 0.09],
                  'iterations'    : [  2000, 3000,4000,5000]
                 }

In [ ]:
Grid_CBC = GridSearchCV(estimator=model_cat, param_grid = parameters, cv = 4, scoring='f1_micro',n_jobs=-1)
Grid_CBC.fit(X_train, y_train, cat_features=categorical_features)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)